In [11]:
import pandas as pd
import win32com.client as win32
import os

In [12]:
#url="https://portafolioinfdoctos.cnbv.gob.mx/Documentacion/minfo/040_13a_R3B.xls"
url="https://portafolioinfdoctos.cnbv.gob.mx/Documentacion/minfo/040_13b_R1.xls"

In [13]:
excel = win32.gencache.EnsureDispatch('Excel.Application')

In [14]:
wb = excel.Workbooks.Open(url)

In [15]:
#Imprimir el nombre de las hojas del libro de excel
for sheet in wb.Worksheets:
    print(sheet.Name)

MINFO
Notas
BD
Formato
Query
Notas T
Calidad de Información


In [16]:
#Eliminar filtro de la tabla dinámica de la hoja "BD" 
pvtTable = wb.Sheets("BD").Range("G6").PivotTable

In [17]:
page_range_item = []
for i in pvtTable.PageRange:
    page_range_item.append(str(i))
    
print(page_range_item)

['cve_destino_credito_grupo', '8.0', 'cve_institucion', '040138', 'cve_periodo', '202112.0']


In [18]:
#Limpiar filtros de la tabla dinámica

filtros=["cve_periodo","cve_institucion","cve_destino_credito_grupo"]

#cve_destino_credito_grupo a excluir
bye = ["8","9"]

for variables in filtros:
    pvtTable.PivotFields(variables).ClearAllFilters()
#Filtrar cve_destino_credito_grupo
for elementos in bye:
    pvtTable.PivotFields("cve_destino_credito_grupo").PivotItems(elementos).Visible = False

#Eliminar la variable id de la tabla dinámica
pvtTable.PivotFields("cve_periodo").Orientation = 1
#Eliminar la variable id de la tabla dinámica
pvtTable.PivotFields("id").Orientation = 0

In [19]:
#Traer los datos de la tabla dinámica a un dataframe
df = pd.DataFrame(pvtTable.TableRange1.Value)
#Fila 1 del dataframe como encabezado
df.columns = df.iloc[1]
#liminar filas 0 y 1
df = df.drop([0,1])
#Eliminar última fila
df = df.drop(df.index[-1])
#Convertir cve_periodo a string
df["cve_periodo"] = df["cve_periodo"].astype(str)
#Crear variables de año y mes
df["año"] = df["cve_periodo"].str[0:4]
df["mes"] = df["cve_periodo"].str[4:6]
#Crear variable de fecha y pasarla a datetime
df["fecha"] = df["año"] + "-" + df["mes"] + "-01"
df["fecha"] = pd.to_datetime(df["fecha"])
#Eliminar variables de año y mes
df = df.drop(["cve_periodo"], axis=1)
df

1,Suma de creditos,Suma de monto,año,mes,fecha
2,31939.0,30131982391.0,2021,12,2021-12-01
3,25132.0,22247216194.0,2021,11,2021-11-01
4,27984.0,22820959830.0,2021,10,2021-10-01
5,29515.0,24420925321.0,2021,09,2021-09-01
6,32438.0,24786322295.0,2021,08,2021-08-01
...,...,...,...,...,...
147,4929.0,3656391473.0,2009,11,2009-11-01
148,5135.0,3903630326.0,2009,10,2009-10-01
149,5486.0,4153916341.0,2009,09,2009-09-01
150,5382.0,4044623811.0,2009,08,2009-08-01


In [20]:
#Cerrar el libro de excel sin guardar los cambios
wb.Close(False)